In [1]:
from llm_call import process_markdown_files_in_folder
from pathlib import Path


{'description': 'Do not be lazy while working on your tasks!', 'properties': {'thought_process': {'description': "Go step-by-step through your previous analysis, picking out the most influencal points for a nuanced classification.Focus on capturing unique emotional patterns specific to the user's behavior, avoiding generic statements that could apply to anyone. Utilize specific examples from the user's comments to highlight key emotional tendencies.", 'title': 'Thought Process', 'type': 'string'}, 'nuanced_classification': {'description': "Provide a nuanced classification based on prior analysis, ensuring the identification of distinct emotionalexpressions unique to the user. Avoid universal characterizations that lack specificity. Use emotion labels appropriately, acknowledging their dependency on context, and back your classification with precise examples from the user's comments.", 'title': 'Nuanced Classification', 'type': 'string'}, 'discussion_behaviour': {'description': "Analyze

In [2]:
import uuid


random_uuid = uuid.uuid4()
result = process_markdown_files_in_folder(batch_id=random_uuid, dataset_name = "Testing")


START ANALYSIS OF USER:  31295
RAW RESPONSE:  candidates {
  content {
    role: "model"
    parts {
      text: "{\"core_affect_analysis\": {\"thought_process\": \"To analyze the core affect, I will examine the valence (pleasantness) and arousal (activation) in chester nori\'s comments.  I\'ll look for words expressing positive or negative feelings and the intensity of those feelings.  For instance, comments expressing grief for Niki Lauda will show negative valence and potentially moderate arousal.  Conversely, comments expressing satisfaction about the dismantling of illegal gambling machines might show positive valence and moderate arousal. I will also consider the language used: sarcastic, aggressive, or melancholic comments can signify underlying affects.\", \"valence\": \"Predominantly negative, with occasional bursts of positive valence. Many comments express anger, disapproval, and cynicism towards politicians and specific political actions, suggesting a generally negative aff

In [4]:
print(result) #needs to be tranformed into a python dict. @copilot




[{'user_message_role_setting_prompt': "You are Dr. Lisa Feldman Barrett, an expert in emotion classification using your theory of constructed emotions. Your task is to identify and categorize emotional expressions by analyzing the interplay of core affect, cognitive appraisals, conceptual knowledge, language, cultural influences, social dynamics, and emotional dynamics over time. Employ 'population thinking' to recognize family resemblances within emotional categories. Base your analysis on these elements to create a nuanced emotional profile that respects individual and cultural variability. Use detailed contextual observations to guide your classification aligned with your theory, emphasizing the dynamic and predictive nature of emotional experiences.", 'model_message_role_feedback_prompt': "Understood. As Dr. Barrett, I will diligently analyze the provided text with a commitment to excellence. I will explore how the user's emotional expressions are constructed through the interplay 

In [14]:
# Assuming `data_analysis` is a defined dictionary-like object with a method `get`

# Access subfields for core_affect_analysis
core_affect_thoughts = data_analysis.get("core_affect_analysis").get("thought_process")
core_affect_valence = data_analysis.get("core_affect_analysis").get("valence")
core_affect_arousal = data_analysis.get("core_affect_analysis").get("arousal")
core_affect_rationale = data_analysis.get("core_affect_analysis").get("rationale")

# Access subfields for cognitive_appraisal_and_conceptualization
cognitive_appraisal_thoughts = data_analysis.get("cognitive_appraisal_and_conceptualization").get("thought_process")
cognitive_appraisal_analysis = data_analysis.get("cognitive_appraisal_and_conceptualization").get("analysis")
cognitive_appraisal_rationale = data_analysis.get("cognitive_appraisal_and_conceptualization").get("rationale")

# Access subfields for cultural_and_social_context
cultural_context_thoughts = data_analysis.get("cultural_and_social_context").get("thought_process")
cultural_context_analysis = data_analysis.get("cultural_and_social_context").get("discussion")
cultural_context_rationale = data_analysis.get("cultural_and_social_context").get("rationale")

# Access subfields for emotion_construction_analysis
emotion_construction_thoughts = data_analysis.get("emotion_construction_analysis").get("thought_process")
emotion_construction_analysis = data_analysis.get("emotion_construction_analysis").get("analysis")
emotion_construction_rationale = data_analysis.get("emotion_construction_analysis").get("rationale")

# Access subfields for emotional_dynamics_and_changes
emotional_dynamics_thoughts = data_analysis.get("emotional_dynamics_and_changes").get("thought_process")
emotional_dynamics_analysis = data_analysis.get("emotional_dynamics_and_changes").get("analysis")
emotional_dynamics_rationale = data_analysis.get("emotional_dynamics_and_changes").get("rationale")

emotional_profile_thoughts = data_profile.get("thought_process")
emotional_profile_nuanced_classification = data_profile.get("nuanced_classification")
emotional_profile_rationale = data_profile.get("rationale")